# Coin Sums
---
### Project Euler # 31

---
In England the currency is made up of pound, £, and pence, p, and there are eight coins in general circulation:

1p, 2p, 5p, 10p, 20p, 50p, £1 (100p) and £2 (200p).

How many different ways can £2 be made using any number of coins?

---
I spent three days trying to understand the mathematical basis for this problem:

https://en.wikipedia.org/wiki/Partition_(number_theory)

http://mathworld.wolfram.com/PartitionFunctionP.html

And then found this general approach rooted in dynamic programming:

https://stackoverflow.com/questions/14992411/understanding-change-making-algorithm

In [1]:
def make_change(target, coins):
    change = [1] + [0 for _ in range(target)]
    for coin in coins:
        for i in range(coin, target + 1):
            change[i] += change[i - coin]
    return change[target]

In [2]:
# US currency - ways to change a dollar
us_currency = [1,5,10,25,50,100]
s = make_change(100, us_currency)
print 'ways to change a dollar using {}: {}'.format(us_currency, s)

ways to change a dollar using [1, 5, 10, 25, 50, 100]: 293


In [3]:
# UK currency - ways to change two pounds
uk_currency = [1,2,5,10,20,50,100,200]
s = make_change(200, uk_currency)
print 'ways to change two pounds using {}: {}'.format(uk_currency, s)

ways to change two pounds using [1, 2, 5, 10, 20, 50, 100, 200]: 73682


In [4]:
%timeit make_change(200, uk_currency)

1000 loops, best of 3: 388 µs per loop


73682 is the right answer and this thing finds it in 391/1000000 of a second on average..

Let's deconstruct this little beast using a simplified US currency example:

How many ways can you make change for 20 cents using only pennies, nickels, and dimes?

In [5]:
target = 20          # 20 cents
currency = [1,5,10]  # pennies, nickels, and dimes

In [6]:
# create a list of 0s whose length equals target + 1 (target value including 0)
change = [1] + [0 for _ in range(target)]
print change

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


These values represent the number of ways to make change for each currency value [i] where:

    0 <= i <= target
    

e.g. the number of ways to make change for 0 cents, 1 cents, 2 cents ... 20 cents

We haven't calculated any combinations yet except for 0:

    there is exactly 1 way to make change for 0 cents... with 0 cents

Then, for each coin in ascending sequence, we calculate the number of combinations that can be made with the current coin and any lesser value coins. 

The number of combinations that can be made with that coin or any combination of lesser coins is given as the current value at that position in the list plus the number of combinations (the list value) that can be made for the value exactly one current coin interval below the current value of interest.

It makes more sense when you see it in action:

In [7]:
# start with the pennies
penny = 1
for i in range(penny, target + 1):  # for every value between 1 and 20
    change[i] += change[i - penny]   # add [i - 1] value to current value
    
print change

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


As expected, the [1] propogates across the entire list as there is exactly one way to make any value with only pennies... i.e. all pennies

In [8]:
# now add the nickels
nickel = 5
for i in range(nickel, target + 1):  # for every value between 5 and 20
    change[i] += change[i - nickel]   # add [i - 5] value to current value
    
print change

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5]


Now it gets interesting:

For values less than 5, there is still only one possible combination: all pennies.

For values between 5 and 9, there are now two possible combinations: all pennies or one nickel and the remainder in pennies.

For values between 10 and 14, there are now three possible combinations: all pennies, one nickel and the remainder in pennies, or two nickels and the remainder in pennies (if any).

For values between 15 and 19, there are now four possible combinations: all pennies, one nickel and the remainder in pennies, two nickels and the remainder in pennies, or three nickels and the remainder in pennies (if any).

Lastly, for value 20, there are now five combinations: all pennies, one nickel and the remainder in pennies, two nickels and the remainder in pennies, three nickels and the remainder in pennies, or four nickels.

For each nickel interval the number of possible combinations is equal to the current list value (representing the number of combinations using only lesser coins - in this case pennies - 1 combination) plus the number of possible combinations calculated at the previous nickel interval.

In this way we can account for the previously calculated combinations and iterativly add new coins and calculate their contribution to the number of possible combinations for a given value.

In [9]:
# add the dimes
dime = 10
for i in range(dime, target + 1):    # for every value between 10 and 20
    change[i] += change[i - dime]   # add [i - 10] value to current value
    
print change

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 9]


Lastly, having added the dimes:

For values less than 10, the number of combinations is unchanged since a dime can't be used to make change for a value less than a dime.

For values between 10 and 14, there is an additional combination available: dime + pennies. The additional combination [1] is encoded in positions 0 - 4 - i.e. positions a dime less.

For values between 15 and 19 there are now two additional combinations: dime + pennies or dime + nickel + pennies (if any). The additional combinations [2] is encoded in positions 5 - 9 - i.e. positions a dime less.

For value 20, there are now four additional combinations: dime + pennies, dime + nickel + pennies, dime + 2 nickels, and 2 dimes. The additional combinations [4] is encoded at position 10 - i.e. the position a dime less.

There are nine ways to make change for 20 cents using pennies, nickels, and dimes.

    1. 20 pennies
    2. 1 nickel,  15 pennies
    3. 2 nickels, 10 pennies
    4. 3 nickels, 5 pennies
    5. 4 nickels
    6. 1 dime, 10 pennies
    7. 1 dime, 1 nickel, 5 pennies
    8. 1 dime, 2 nickels
    9. 2 dimes

In [10]:
# success!